In [1]:
import sys
sys.path.append('../src')
sys.path.append('../src/osqp')

from utils import *
from osqp_solver import *
from osqp_sqp import *
from osqp_mpc import *
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf
import numpy as np
np.set_printoptions(linewidth=99999999)
    
model_dir = "../description/"
urdf_path = model_dir + "indy7.urdf"
model, visual_model, collision_model = pin.buildModelsFromUrdf(urdf_path, model_dir)

# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()

# Create MPC solver and optimizer
solver = OSQPSolver(model)
sqp = SQP_OSQP(solver)
controller = MPC_OSQP(model, sqp, solver)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7035/static/


In [ ]:


q = np.array([0.5, 0.6, -2.0, -1.0, -1.0, 0.1])

# Forward kinematics to get end effector position
pin.forwardKinematics(model, data, q)
ee_pos = data.oMi[6].translation # Joint 6 is the end effector
print(f"End effector position: {ee_pos}")

viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(open=True)
viz.loadViewerModel(rootNodeName="robot")

while True:
    viz.display(q)

In [2]:
# initial state - zeros
xstart = np.ones(solver.nx)

# goal ee positions
endpoints = np.array([
    #np.array(controller.eepos(np.zeros(controller.nq))),
    np.array(solver.eepos(0.3*np.ones(6))),
    np.array(solver.eepos(0.9*np.ones(6))),
    #np.array(controller.eepos(np.zeros(controller.nq))), 
    np.array(solver.eepos(np.array([-3.0, 1.0, 1.0, 1.0, 0.0, 0.0])))
])

print("Endpoints:")
print(endpoints)

# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)

Endpoints:
[[-0.34013996 -0.30723899  1.15448739]
 [-0.33030096 -0.77069748  0.53591299]
 [ 0.89016846  0.23030064  0.46709395]]


In [3]:
# Run MPC
q_trajectory = controller.run_mpc(xstart, endpoints)

0.8822365440884666
0.8932961725209858
0.9053590870084658
0.9169408371914417
0.9255210446352959
0.9282393131206288
0.9225972253239547
0.9066562250253478
0.8807399644219697
0.8476509587247106
0.809491369348472
0.7677095706008326
0.7228396855961605
0.6758283420274132
0.6277886425864444
0.5796551655302516
0.5323310640845136
0.4867107126024096
0.4437596939585305
0.4040917559222529
0.36816537555202
0.3366494243755835
0.31016925915217247
0.28942801494707626
0.2747799069469508
0.2662859174759684
0.2636137187158445
0.2661209487991101
0.27287892017457693
0.2827799203353352
0.29472704639164643
0.30792313663720616
0.32140517860882856
0.3342395813650952
0.34644660264187743
0.35824137143012763
0.3695792088324464
0.3803384502251258
0.3905298876607012
0.40007179045992025
0.40887900235887664
0.416898013443405
0.4240984754578205
0.43035558549210895
0.4357045703598418
0.4401263107676622
0.44380690540169915
0.4466944723057237
0.44875179278446303
0.4501118113773361
0.45096636741262947
0.451171901614343
0.4

In [4]:
print_stats(sqp.get_stats())
viz.play(q_trajectory=q_trajectory, dt=0.01)

qp_iters:
  avg: 2.00 
  min: 2.00 
  max: 2.00 

linesearch_alphas:
  avg: 0.54 
  min: 0.00 
  max: 1.00 

sqp_stepsizes:
  avg: 70.57 
  min: 2.82 
  max: 611.99 



In [7]:
# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()
# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)
#print_stats(controller.solver.get_stats())
while 1:
    viz.play(q_trajectory=q_trajectory, dt=0.01)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7038/static/


KeyboardInterrupt: 